In [465]:
import pandas as pd 
import numpy as np
import seaborn as sns
import random

In [466]:
%matplotlib inline

In [467]:
### some variables will have the 'portuguese' name attuned to it

In [468]:
casas = pd.read_csv('housing.csv')

In [469]:
casas.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [470]:
len(casas)

20640

In [471]:
casas1 = casas[casas.ocean_proximity == '<1H OCEAN']
casas2 = casas[casas.ocean_proximity == 'INLAND']
casas = pd.concat([casas1,casas2], axis=0)

In [472]:
## Por aqui verificamos que ha valores missing na coluna total_bedrooms
casas.isna().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        157
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

## Question 2

In [473]:
## here we get the 50% pencetile of the population -> 1195
casas.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,15687.000000,15687.000000,15687.000000,15687.000000,15530.000000,15687.000000,15687.000000,15687.000000,15687.000000
mean,-119.217442,35.467307,27.188118,2665.677312,541.244688,1466.317205,500.916491,3.804019,191943.020017
std,1.780380,2.066387,12.057128,2257.672743,435.650018,1180.389908,392.759399,1.857158,108801.077620
min,-124.140000,32.610000,1.000000,2.000000,2.000000,3.000000,2.000000,0.499900,14999.000000
25%,-121.070000,33.940000,17.000000,1441.000000,295.000000,802.000000,278.000000,2.520800,111300.000000
50%,-118.370000,34.160000,27.000000,2118.000000,432.000000,1195.000000,406.000000,3.468800,166900.000000
75%,-117.990000,37.340000,36.000000,3172.000000,645.000000,1777.000000,602.000000,4.686200,241100.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


### here I sample all the values and create variables with specific sizes, also dividing the original dataframe into three

In [474]:
casas_random = casas.sample(frac=1.0,random_state=42)

In [475]:
casas_random.median_house_value = np.log1p(casas_random.median_house_value)

In [476]:
casas_random.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
19752,-122.22,40.18,13.0,3719.0,803.0,1754.0,764.0,2.3517,11.395279,INLAND
8325,-118.29,33.94,38.0,2407.0,630.0,1774.0,562.0,1.5615,11.595436,<1H OCEAN
1148,-121.36,39.52,15.0,2490.0,527.0,1229.0,497.0,2.3917,11.358620,INLAND
8893,-118.52,34.01,25.0,2757.0,738.0,1014.0,633.0,3.1433,13.122367,<1H OCEAN
13084,-121.36,38.57,26.0,1793.0,244.0,653.0,235.0,5.6485,11.771444,INLAND


In [477]:
sizeData = len(casas_random)
val_size = int(0.2*sizeData)
test_size = int(0.2*sizeData)
train_size = sizeData - val_size - test_size

In [478]:
valData = casas_random.iloc[:val_size]
testData = casas_random.iloc[val_size:val_size+test_size]
trainData = casas_random.iloc[val_size+test_size:]

### the values of median_house_value will be in logarithmic base

## Question 3

In [479]:
## filling the values NaN with zero
casas_filled_zeros = trainData.copy()
casas_filled_zeros.total_bedrooms = casas_filled_zeros.total_bedrooms.fillna(0)
column_indices = [i for i, col in enumerate(casas_filled_zeros.columns) if col != 'ocean_proximity']
casas_filled_zeros = casas_filled_zeros.iloc[:, column_indices]
X_train_zeros = casas_filled_zeros.values

In [480]:
casas_filled_zeros.isna().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
dtype: int64

In [481]:
## filling the values NaN with the mean value
casas_filled_mean = trainData.copy()
bedroomsmean = casas_filled_mean.total_bedrooms[casas_filled_mean.total_bedrooms.notna()].mean()
casas_filled_mean.total_bedrooms = casas_filled_mean.total_bedrooms.fillna(bedroomsmean)
casas_filled_mean = casas_filled_mean.iloc[:, column_indices]
X_train_mean = casas_filled_mean.values

In [482]:
casas_filled_mean.isna().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
dtype: int64

In [483]:
def train_linear_reg(X,Y):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones,X])
    XTX = X.T@X
    XTX_inv = np.linalg.inv(XTX)
    w_full = XTX_inv@X.T@(Y)
    
    return w_full[0], w_full[1:]
    

#### using zeros

In [484]:
y_train_1 = list(casas_filled_zeros.median_house_value)
w0_zero, w_zero = train_linear_reg(X_train_zeros,y_train_1)
w_zero
y_pred1 = w0_zero + X_train_zeros@w_zero


#### using the mean

In [485]:
y_train_2 = list(casas_filled_mean.median_house_value)
w0_mean, w_mean = train_linear_reg(X_train_mean,casas_filled_mean.median_house_value)
w_mean
y_pred2 = w0_mean + X_train_mean@w_mean

## Functions

In [486]:
def rmse(y, y_pred):
    error = (y - y_pred)**2
    meansq = error.mean()
    return np.sqrt(meansq)

In [487]:
def preparing_X(df):
    #colq = list(df.columns[df.isna().any()])
    copia = df.copy()
    copia.total_bedrooms = copia.total_bedrooms.fillna(0)
    column_indices = [i for i, col in enumerate(copia.columns) if col != 'ocean_proximity']
    copia = copia.iloc[:, column_indices]
    return copia.values

In [488]:
X_test1 = preparing_X(testData)
X_val1 = preparing_X(valData)

y_pred_test1 = w0_zero + X_test1@w_zero
y_pred_val1 = w0_zero + X_val1@w_zero
round(rmse(y_pred_test1,y_pred_val1),2)

0.79

In [489]:
y_pred_test2 = w0_mean + X_test1@w_mean
y_pred_val2 = w0_mean + X_val1@w_mean
round(rmse(y_pred_test2,y_pred_val2),2)


0.79

## Question 4

In [490]:
r = [0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10]

In [491]:
def train_linear_reg_regulized(X,Y,r):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones,X])
    
    XTX = X.T@(X)
    XTX = XTX + r*np.eye(XTX.shape[0])
    XTX_inv = np.linalg.inv(XTX)
    w_full = XTX_inv@X.T@(Y)
    
    return w_full[0], w_full[1:]
    

In [492]:
rmseValues = np.zeros(len(r))
for i in range(len(r)):
    w0_zero, w_zero = train_linear_reg_regulized(X_train_zeros,casas_filled_zeros.median_house_value,i)
    w_zero
    y_pred1 = w0_zero + X_train_zeros@w_zero
    X_test1 = preparing_X(testData)
    X_val1 = preparing_X(valData)
    y_pred_test1 = w0_zero + X_test1@w_zero
    y_pred_val1 = w0_zero + X_val1@w_zero
    rmseValues[i] = rmse(y_pred_test1,y_pred_val1)
    

In [493]:
list(rmseValues).index(rmseValues.max())


0

## Question 5

In [494]:
random_seeds = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [495]:
standard_deviation_seeds = np.zeros(10)
for i in range(10):
    casas_random = casas.sample(frac=1.0,random_state=i)
    casas_random.median_house_value = np.log1p(casas_random.median_house_value)
    sizeData = len(casas_random)
    val_size = int(0.2*sizeData)
    test_size = int(0.2*sizeData)
    train_size = sizeData - val_size - test_size
    valData = casas_random.iloc[:val_size]
    testData = casas_random.iloc[val_size:val_size+test_size]
    trainData = casas_random.iloc[val_size+test_size:]
    ## filling the values NaN with zero
    casas_filled_zeros = trainData.copy()
    casas_filled_zeros.total_bedrooms = casas_filled_zeros.total_bedrooms.fillna(0)
    column_indices = [i for i, col in enumerate(casas_filled_zeros.columns) if col != 'ocean_proximity']
    casas_filled_zeros = casas_filled_zeros.iloc[:, column_indices]
    X_train_zeros = preparing_X(trainData)	
    X_test1 = preparing_X(testData)
    X_val1 = preparing_X(valData)
    y_train_1 = list(casas_filled_zeros.median_house_value)
    w0_zero, w_zero = train_linear_reg(X_train_zeros,y_train_1)
    y_pred_test1 = w0_zero + X_test1@w_zero
    y_pred_val1 = w0_zero + X_val1@w_zero
    y_pred_train1 = w0_zero + X_train_zeros@w_zero
    standard_deviation_seeds[i] = round(rmse(y_pred_test1,y_pred_val1),6)
    
round(np.std(standard_deviation_seeds),3)


0.01

## Question 6

In [496]:
casas_random = casas.sample(frac=1.0,random_state=9)

In [497]:
casas_random.median_house_value = np.log1p(casas_random.median_house_value)

In [498]:
sizeData = len(casas_random)
val_test_size = int(0.4*sizeData)
train_size = sizeData - val_test_size

In [499]:
valTestData = casas_random.iloc[:val_test_size]
trainData = casas_random.iloc[val_test_size:]

In [500]:
## filling the values NaN with zero
casas_filled_zeros = trainData.copy()
casas_filled_zeros.total_bedrooms = casas_filled_zeros.total_bedrooms.fillna(0)
column_indices = [i for i, col in enumerate(casas_filled_zeros.columns) if col != 'ocean_proximity']
casas_filled_zeros = casas_filled_zeros.iloc[:, column_indices]
X_train_zeros = casas_filled_zeros.values

In [501]:
valTestDataPrepared = preparing_X(valTestData)
testando = valTestData.median_house_value.values

In [508]:
y_train_1 = list(trainData.median_house_value)
w0_zero, w_zero = train_linear_reg_regulized(X_train_zeros,y_train_1,0.001)
y_test1 = w0_zero + valTestDataPrepared@w_zero

12.07254696717504

In [504]:
rmse(y_test1,testando)
# operands could not be broadcast together with shapes (9413,) (6274,) ???

3.2678254063927675e-07